In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np

In [ ]:
adata = sc.read_h5ad('GSE174188_CLUES1_adjusted.h5ad')
adata

In [ ]:
adata = adata.raw.to_adata()
# Filter out cells with fewer than 500 counts
sc.pp.filter_cells(adata, min_counts=500)

# Filter out cells that express fewer than 200 genes
sc.pp.filter_cells(adata, min_genes=200)

# Filter out genes expressed in fewer than 5% of cells
sc.pp.filter_genes(adata, min_cells=int(0.05 * adata.shape[0]))
adata

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('MT-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]
adata

In [ ]:
# Display all cell types
cell_types = adata.obs['cg_cov'].unique()
cell_types

In [ ]:
# Subset data by cell type
cell_subset = adata[adata.obs['cg_cov'] == 'B']
cell_subset

In [ ]:
pbs = []

for sample in cell_subset.obs.ind_cov.unique():
    # Filter cells based on the sample ID
    samp_cell_subset = cell_subset[cell_subset.obs['ind_cov'] == sample]
    
    # Sum the total counts for each gene across all cells within the same pseudobulk
    total_counts_per_gene = samp_cell_subset.X.sum(axis=0)

    pseudobulk_counts = pd.DataFrame(index=cell_subset.var_names)
    # Append the total counts to the pseudobulk_counts DataFrame
    pseudobulk_counts[sample] = total_counts_per_gene.A1  # A1 is used to flatten the matrix to 1D array

    # Sum all the counts for all genes in the array
    total_counts_all_genes = pseudobulk_counts[sample].sum(axis=0)

    # # Normalize the counts for each gene by dividing the count for each gene by the total count for all genes in the pseudobulk
    normalized_counts = pseudobulk_counts.divide(total_counts_all_genes)

    # Calculate Counts Per Million (CPM)
    CPM = normalized_counts * 1_000_000

    # Log2-transform the CPM values after adding 1 to avoid log(0) issues
    log2_CPM = np.log2(CPM + 1)

    # Create a new AnnData object with log2_CPM
    rep_adata = sc.AnnData(X=log2_CPM.T,
                           var=samp_cell_subset.var[[]])

    # Assign the sample ID as the observation name
    rep_adata.obs_names = [sample]

    # Assign the 'SLE_status' annotation based on the first cell's 'SLE_status' in the current sample
    rep_adata.obs['SLE_status'] = samp_cell_subset.obs['SLE_status'].iloc[0]

    # Append the representative AnnData object to the list pbs
    pbs.append(rep_adata)

In [ ]:
pb = sc.concat(pbs)

In [ ]:
# View pseudobulk samples
pb
pb.obs
pb.var

In [ ]:
# Perform differential analysis
sc.tl.rank_genes_groups(pb, groupby='SLE_status', method='wilcoxon', groups=['SLE', 'Healthy'])

In [ ]:
# Access the result
result_df = sc.get.rank_genes_groups_df(pb, group='Healthy', key='rank_genes_groups')
result_df.to_csv('B_pseudobulk_result.tsv', sep='\t', index=False)